# Step30, 고차 미분(준비 편), 복습편


현재 DeZero는 미분을 자동으로 계산할 수 있지만 1차 미분 한정이다.  
2차 미분 나아가 3차 미분, 4차, 5차 등등 형태의 무든 고차 미분까지 자동으로 계산할 수 있도록 DeZero를 확장해본다.

그러려면 DeZero를 사용하여 2차 미분을 계산하려는 현재의 역전파 구현을 근본적으로 재검토해야 한다.  
DeZero의 역전파는 Variable과 Function 클래스에 기초해 동작하기 때문에 Variable과 Function의 현재 구현을 되돌아본다.


- 계산의 '연결'은 Function 클래스의 \__call\__메서드에서 만들어진다.
- 구체적인 순전파와 역전파 계산은 Function 클래스를 상속한 클래스의 forward 메서드와 backward 메서드로 처리한다.

# Step31, 고차 미분(이론 편)

이전까지의 구현에서 주목할 점은 계산 그래프의 '연결'이 만들어지는 시점으로, 순전파를 계산할 때 만들어진다.  
역전파를 계산할 때는 만들어지지 않는데 여기에 문제의 핵심이 있다.  

## 31.1 역전파 계산 

순전파와 마찬가지로 역전파에도 구체적인 계산 로직이 있다.  
역전파를 할때 계산에서 ndarray 인스턴스가 사용되면 아무런 계산 그래프도 만들지 않는다.

만약 역전파를 계산할 때도 '연결'이 만들어졌을 때 장점  
--> 고차 미분을 자동으로 계산할 수 있게 된다.

앞으로의 목표는 '미분 계산'을 계산 그래프로 만드는 것이다.  
여기서 '미분 계산' : 역전파 때 수행되는 계산.  
따라서 역전파 때 수행되는 계산에 대해서도 '연결'을 만들면 문제가 해결된다.

## 31.2 역전파로 계산 그래프 만들기 

DeZero는 순전파 계산의 '연결'을 만든다.  
'Variable 인스턴스를 사용'하여 일반적인 계산 (순전파)을 하는 시점에 '연결'이 만들어진다.  
즉 함수의 backward 메서드에서도 ndarray 인스턴스가 아닌 Variable 인스턴스 형태로 유지해야한다.

지금까지 Variable 클래스의 grad는 ndarray 인스턴스를 참조했다.  
이를 Variable 인스턴스를 참조하도록 변경한다.

중요한 점은 역전파 계산에 대한 계산 그래프도 만들어진다는 것이다.  
미분값을 나타내는 gy가 Variable 인스턴스가 된 덕분에 gy를 사용한 계산에도 '연결'이 만들어지는 것이다.

y.backward()을 호출함으로써 '새로' 만들어지는 계산 그래프. 즉, '역전파에 의해 새롭게 계산 그래프가 만들어졌다'

# Step32, 고차 미분(구현 편)

역전파 시 수행되는 계산에 대해서도 계산 그래프를 만들면 된다.  
역전파 때도 Variable 인스턴스를 사용하면 해결된다.

## 32.1 새로운 DeZero

dezero 폴더의 core.py를 만들어 새로 만든다.

~~~python 
class Variable:

    def backward(self, retain_grad=False):      # 추가되는곳 : 
        if self.grad is None:
            # self.grad = np.ones_like(self.data)     # 미분값이 없으면 모두 1로 구성된 행렬
            self.grad = Variable(np.ones_like(self.data))
~~~

단 한줄 이 작은 변경덕에 미분값을 자동으로 저장하는 코드에서 self.grad가 Variable 인스턴스를 담게된다. 이상으로 새로운 Variable 클래스를 완성했다.


## 32.2 함수 클래스의 역전파 

Function 클래스는 수정할 게 없다.  
구체적인 함수들의 backward 메서드를 수정해야한다.  
- ADD
- Mul
- Neg 
- Sub 
- Div 
- Pow 

**Add**
~~~python 
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

~~~
Add 클래스의 역전파가 하는 일은 출력 쪽에서 전해지는 미분값을 입력 쪽으로 전달하는게 다이다.  
이처럼 역전파 때는 아무것도 계산하지 않기 때문에 수정할 거리가 없다.

**Mul**
~~~python
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        # x0, x1 = self.inputs[0].data, self.inputs[1].data
        x0, x1 = self.inputs
        return gy * x1, gy * x0
~~~
수정 전에는 Variable 인스턴스 안에 있는 데이터(ndarray 인스턴스)를 꺼내야 했다.  
수정 후에는 Mul 클래스에서 Variable 인스턴스를 그대로 사용한다.

역전파를 계산하는 gy * x1 코드를 보자. 새로운 DeZero에서는 gy와 x1이 Variable 인스턴스이다.  
gy * x1이 실행되는 뒤편에서는 Mul 클래스의 순전파가 호출된다.  
그때 Function.\__call\__()이 호출되고 그 안에서 계산 그래프가 만들어진다.


## 32.3 역전파를 더 효율적으로(모드 추가)

18step에서 역전파의 활성/비활성 모드를 도입했었다. 역전파가 필요없는 경우에는 '역전파 비활성 모드'로 전환하여 역전파 처리를 생략한 것이다.


역전파를 처음 한 번 이후로 다시 할 일이 없다면 역전파 계산도 '역전파 비활성 모드'로 실행하도록 한다.

~~~python 
def backward(self, retain_grad=False, create_graph=False):

    # ....생략.....

        while funcs:
            f = funcs.pop()                         # 함수들을 하나씩 뽑는다.
            gys = [output().grad for output in f.outputs]     # 출력변수인 outputs에 담겨있는 미분값(.grad)들을 리스트에 담는다
            
            with using_config('enable_backprop', create_graph):
                gxs = f.backward(*gys)              # 메인 backward # f의 역전파를 호출한다. *를 붙혀 리스트를 풀면서 넣어준다.(리스트 언팩)
                if not isinstance(gxs, tuple):                  # gxs가 튜플이 아니면 튜플로 변환한다.
                    gxs = (gxs,)

                for x, gx in zip(f.inputs, gxs):                # gxs와 f.inputs의 각 원소는 서로 대응 관계
                    if x.grad is None:
                        x.grad = gx                             # 역전파로 전파되는 미분값을 Variable의 인스턴스 변수 grad에 저장
                    else:
                        x.grad = x.grad + gx    # x.grad가 None이 아니라 기존에 가지고 있는 값이 있다면 가지고 있는 값에 gx를 추가로 더한다.

                    if x.creator is not None:
                        add_func(x.creator)      # <-- 바뀐부분, 수정전: funcs.append(x.creator) 출처가 있는 데이터를 add_funcs에 넣는다.
            
  # ....생략.....
~~~

create_graph가 False이면 역전파로 인한 계산은 '역전파 비활성 모드'에서 이루어진다.  

또한 create_graph=False로 기본 설정한 이유는 실무에서 역전파가 단 1회만 수행되는 경우가 압도적으로 많기 때문이다.  
만약 2차 이상의 미분이 필요하면 create_graph를 True로 설정한다.  
그러면 역전파를 계산할 때도 계산 그래프가 만들어지므로 역전파를 반복해서 할 수 있다.

**NOTE_**
가령 Mul 클래스의 backward 메서드는 gy * x1 계산을 한다. '*연산자'는 오버로드 되어 있어 실제로는 Mul()(gy,x1) 코드가 호출되고  
이어서 부모 클래스인 Function의 \__call\__()이 호출된다.  
Function.\__call\__() 메서드에서 Config.enable_backprop이 참조되고, 역전파 활성/비활성 모드가 전환된다.



## 32.4 \__init\__.py 변경



In [1]:
is_simple_core = False  # True
# =============================================================================

if is_simple_core:
    from dezero.core_simple import Variable
    from dezero.core_simple import Function
    from dezero.core_simple import using_config
    from dezero.core_simple import no_grad
    from dezero.core_simple import as_array
    from dezero.core_simple import as_variable
    from dezero.core_simple import setup_variable
else:
    from dezero.core import Variable
    from dezero.core import Function
    from dezero.core import using_config
    from dezero.core import no_grad
    from dezero.core import as_array
    from dezero.core import as_variable
    from dezero.core import setup_variable


setup_variable()